### Import Necessary Modules

In [1]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

### Load Data

In [ ]:
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()

print("Training Data Shape:", trainX.shape)
print("Test Data Shape:", testX.shape)

### Preprocess Data

In [3]:
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

trainY = to_categorical(trainY, num_classes=10)
testY = to_categorical(testY, num_classes=10)

trainX_cnn = np.expand_dims(trainX, axis=-1)
testX_cnn = np.expand_dims(testX, axis=-1)

trainX_fcnn = trainX.reshape(trainX.shape[0], -1)
testX_fcnn = testX.reshape(testX.shape[0], -1)

### Train & Test FCNN (Define FCNN)

In [ ]:
def build_fcnn(input_shape, num_classes):
    from tensorflow.keras.layers import Input, Dense
    from tensorflow.keras.models import Model

    inputs = Input(shape=input_shape, name='InputLayer')
    x = Dense(128, activation='relu', name='DenseLayer1')(inputs)
    x = Dense(64, activation='relu', name='DenseLayer2')(x)
    outputs = Dense(num_classes, activation='softmax', name='OutputLayer')(x)
    model = Model(inputs, outputs, name='FullyConnectedNN')
    model.summary()
    return model

fcnn = build_fcnn((784,), 10)

### Compile & Train

In [ ]:
fcnn.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_fcnn = fcnn.fit(trainX_fcnn, trainY, validation_split=0.1, epochs=10, batch_size=32)

### Test FCNN

In [ ]:
test_loss_fcnn, test_acc_fcnn = fcnn.evaluate(testX_fcnn, testY)
print(f"FCNN Test Accuracy: {test_acc_fcnn}")

### Train & Test CNN (Define CNN)

In [ ]:
def build_cnn(input_shape, num_classes):
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
    from tensorflow.keras.models import Model

    inputs = Input(shape=input_shape, name='InputLayer')
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', name='ConvLayer1')(inputs)
    x = MaxPooling2D(pool_size=(2, 2), name='MaxPool1')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', name='ConvLayer2')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='MaxPool2')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='DenseLayer1')(x)
    outputs = Dense(num_classes, activation='softmax', name='OutputLayer')(x)
    model = Model(inputs, outputs, name='ConvolutionalNN')
    model.summary()
    return model

cnn = build_cnn((28, 28, 1), 10)

### Compile & Train CNN

In [ ]:
cnn.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_cnn = cnn.fit(trainX_cnn, trainY, validation_split=0.1, epochs=10, batch_size=32)

### Test CNN

In [ ]:
test_loss_cnn, test_acc_cnn = cnn.evaluate(testX_cnn, testY)
print(f"CNN Test Accuracy: {test_acc_cnn}")

### Compare and Discuss Results (Plot Training Accuracy)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history_fcnn.history['accuracy'], label='FCNN Training Accuracy')
plt.plot(history_fcnn.history['val_accuracy'], label='FCNN Validation Accuracy')
plt.plot(history_cnn.history['accuracy'], label='CNN Training Accuracy')
plt.plot(history_cnn.history['val_accuracy'], label='CNN Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()